# Looking for the objects in the catalogs

We will try to find the onjects from the lst in the catalogs TeV, GeV and Simbad.

- PSR B1259-63
- LS 5039
- LS   I +61  132
- HESS J0632+057
- PSR J2032+4127
- 1FGL J1018.6-5856
- HESS J0536-675


In [1]:
import pandas as pd
import re
import sys

In [2]:
regular_expressions = ['.*B12.*63.*',
                       '.*LS.*5039.*',
                       '.*LS.*I.*61.*303.*',
                       '.*J0632.*057.*',
                       '.*J2032.*.*4127.*',
                       '.*1FGL.*J1018.*5856.*',
                       '.*J0536.*67.*',
                      ]

This cell loads data created in simbad.ipynb

In [38]:
tevsimbad = pd.read_csv("data/tevsimbad.txt", sep='\t', index_col=0)
gevsimbad = pd.read_csv("data/gevsimbad.txt", sep='\t', index_col=0)
gevtevsimbad = pd.read_csv("data/gevtevsimbad.txt", sep='\t', index_col=0)
pd_gev = pd.read_csv("data/gev.txt", sep='\t', index_col=0)
pd_tev = pd.read_csv("data/tev.txt", sep='\t', index_col=0)
pd_simbad = pd.read_csv("data/simbad.txt", sep='\t', index_col=0)
pd_gevtev_simbadclasses = pd.read_csv("data/gevtev_simbadclasses.txt", sep='\t', index_col=0)
pd_gevtev_simbadclasses = pd_gevtev_simbadclasses.loc[:, ~pd_gevtev_simbadclasses.columns.str.contains('^Unnamed')]
pd_gevtev_simbadclasses = pd_gevtev_simbadclasses.reset_index(drop=True)
pd_gevtev_simbadclasses.to_csv("data/gevtev_simbadclasses.txt", sep='\t')
pd_gevtevxmm_simbadclasses = pd.read_csv('data/gevtev_simbadclasses_xmm.txt',sep = '\t', index_col=0)
pd_gevtevxmm_simbadclasses = pd_gevtevxmm_simbadclasses.loc[:, ~pd_gevtevxmm_simbadclasses.columns.str.contains('^Unnamed')]
pd_gevtevxmm_simbadclasses = pd_gevtevxmm_simbadclasses.reset_index(drop=True)
pd_gevtevxmm_simbadclasses.to_csv('data/gevtev_simbadclasses_xmm.txt', sep='\t')

In [40]:
print(len(pd_gevtev_simbadclasses))
print(len(pd_gevtevxmm_simbadclasses))

111
5067


The following cell will be useful to determine a string name of a catalog where an object is found. It is not the cleverest way to do it, as if number of elements is changed(another epsilon is used), this dictionary should be corrected manually.

In [41]:
len_catalogs = {len(tevsimbad) : 'tevsimbad',
            len(gevsimbad) :'gevsimbad',
            len(gevtevsimbad) :'gevtevsimbad',
            len(pd_gev) : 'gev',
            len(pd_tev) : 'tev',
            len(pd_simbad) : 'simbad',
            len(pd_gevtev_simbadclasses) : 'gevtev_simbadclasses',
            len(pd_gevtevxmm_simbadclasses) : 'pd_gevtevxmm_simbadclasses'
               }

In [42]:
name_to_catalog = {'tevsimbad' : tevsimbad,
            'gevsimbad' : gevsimbad,
            'gevtevsimbad' : gevtevsimbad,
            'gev' : pd_gev,
            'tev' : pd_tev,
            'simbad' : pd_simbad,
            'gevtev_simbadclasses' : pd_gevtev_simbadclasses,
            'pd_gevtevxmm_simbadclasses' : pd_gevtevxmm_simbadclasses}

In [43]:
gev_columns = ['gev_2FGL_Name', 'gev_1FGL_Name', 
                 'gev_1FHL_Name', 'gev_2FGL_Name', 
                 'gev_ASSOC1', 'gev_ASSOC2',
                'gev_ASSOC_GAM1', 'gev_ASSOC_GAM2',
                'gev_ASSOC_GAM3', 'gev_ASSOC_TEV', 'gev_CLASS1',
                'gev_GLAT', 'gev_GLON']

tev_columns = ['tev_common_name', 'tev_gamma_names', 
                'tev_fermi_names', 'tev_other_names', 
                'tev_classes',
                'tev_glat', 'tev_glon']

s_columns = ['s_class', 's_spectra', 's_glat', 's_glon',
            's_MAIN_ID']

xmm_columns = ['xmm_IAUNAME']

s_tev_columns = tev_columns + s_columns
s_gev_columns = gev_columns + s_columns
tev_gev_columns = gev_columns + tev_columns
tev_gev_columns_s = gev_columns + tev_columns + ['s_class', 's_simbad', 's_MAIN_ID']
gevtevs_columns = gev_columns + tev_columns + s_columns
tev_gev_xmm_columns_s = gev_columns + tev_columns + xmm_columns + ['s_class', 's_simbad', 's_MAIN_ID']

In [44]:
name_to_column = {'tevsimbad' : s_tev_columns,
            'gevsimbad' : s_gev_columns,
            'gevtevsimbad' : gevtevs_columns,
            'gev' : gev_columns,
            'tev' : tev_columns,
            'simbad' : s_columns,
            'gevtev_simbadclasses' : tev_gev_columns_s,
            'pd_gevtevxmm_simbadclasses' : tev_gev_xmm_columns_s}

In [45]:
def find_in_catalogs(string_to_find, catalog):
    """Looks for an object containing in its name string_to_find in the catalog
    
    Inputs:
    string_to_find - string containing a part og a name that we are looking for
    catalog - in which catalog to look for (pandas DataFrame)
    """
    for c in catalog.columns:
        for obj in catalog[c]:
            if (str(obj)).find(string_to_find) != -1:
                print(len_catalogs[len(catalog)] + ' :')
                print('\t' + str(c))
                print('\t\t' + (str(obj)))
            


In [46]:
def old_find_number_in_catalogs(string_to_find, catalog, printRes = False):
    #Based on string find
    """Looks for an object containing in its name string_to_find in the catalog,
    Finds its number in the corresponging catalog
    
    Inputs:
    string_to_find - string containing a part og a name that we are looking for
    catalog - in which catalog to look for (pandas DataFrame)
    printRes - True if you want to see the search
    
    Returns:
    success - if found is equal to 1, if not - 0
    catalog_name - name of the catalog where string was found(string)
    number_obj - number of a corresponding line
    
    """
    success = 0
    catalog_name = '' 
    number_obj = -1
    for c in catalog.columns:
        for number_obj in range(len(catalog[c])):

            obj = catalog[c][number_obj]
            if (str(obj)).find(string_to_find) != -1:
                success = 1
                
                catalog_name = len_catalogs[len(catalog)]
                
                if printRes:
                    print(len_catalogs[len(catalog)] + ' :')
                    print('\t' + str(c))
                    print('\t\t' + (str(obj)))
                    print('\t\t\t' + str(number_obj))
                return success, catalog_name, number_obj
                
    return success, catalog_name, number_obj

In [47]:
def find_number_in_catalogs(regular_expression, catalog, printRes = False):
    #based on regular expressions
    """Looks for an object containing in its name string_to_find in the catalog,
    Finds its number in the corresponging catalog
    
    Inputs:
    string_to_find - string containing a part og a name that we are looking for
    catalog - in which catalog to look for (pandas DataFrame)
    printRes - True if you want to see the search
    
    Returns:
    success - if found is equal to 1, if not - 0
    catalog_name - name of the catalog where string was found(string)
    number_obj - number of a corresponding line
    
    """
    success = 0
    catalog_name = '' 
    number_obj = -1
    for c in catalog.columns:
        for number_obj in range(len(catalog[c])):
            try:
                obj = catalog[c][number_obj]
            except:
                print(c)
                display(catalog[c])
                print(number_obj)
            if (re.search(regular_expression, str(obj)) is not None):
                success = 1
                
                catalog_name = len_catalogs[len(catalog)]
                
                if printRes:
                    print(len_catalogs[len(catalog)] + ' :')
                    print('\t' + str(c))
                    print('\t\t' + (str(obj)))
                    print('\t\t\t' + str(number_obj))
                return success, catalog_name, number_obj
                
    return success, catalog_name, number_obj

In [54]:
catalogs = [#gevtevsimbad,
            #tevsimbad,
            #gevsimbad,
            pd_gevtev_simbadclasses,
            pd_gevtevxmm_simbadclasses
           ]

In [55]:
separate_catalogs = [
    pd_gev,
    pd_tev,
    pd_simbad,
]

Strings we want to find in the catalogs:

In [56]:
strings_to_find = [
    'PSR B1259-63',
    'LS 5039',
    'LS   I +61  303',
    'HESS J0632+057',
    'LAT PSR J2032+4127',
    '1FGL J1018.6-5856',
    'J0536',
    
]

In [57]:
resultats_of_search = {}
#this dictionary remembers catalog name and corresponding number of line where an interesting objects is found
for i in len_catalogs.values():
    resultats_of_search.update({i : []})

In [58]:
for regular_expression in regular_expressions:
    print('Looking for : ' + regular_expression)
    for j in catalogs:

        success, catalog_name, number_obj = find_number_in_catalogs(regular_expression, j)
        print(len_catalogs[len(j)])
        if success:
            resultats_of_search[catalog_name].append(number_obj)
print('Done')

Looking for : .*B12.*63.*
gevtev_simbadclasses
pd_gevtevxmm_simbadclasses
Looking for : .*LS.*5039.*
gevtev_simbadclasses
pd_gevtevxmm_simbadclasses
Looking for : .*LS.*I.*61.*303.*
gevtev_simbadclasses
pd_gevtevxmm_simbadclasses
Looking for : .*J0632.*057.*
gevtev_simbadclasses
pd_gevtevxmm_simbadclasses
Looking for : .*J2032.*.*4127.*
gevtev_simbadclasses
pd_gevtevxmm_simbadclasses
Looking for : .*1FGL.*J1018.*5856.*
gevtev_simbadclasses
pd_gevtevxmm_simbadclasses
Looking for : .*J0536.*67.*
gevtev_simbadclasses
pd_gevtevxmm_simbadclasses
Done


In [59]:
print(resultats_of_search)

{'tevsimbad': [], 'gevsimbad': [], 'gevtevsimbad': [], 'gev': [], 'tev': [], 'simbad': [], 'gevtev_simbadclasses': [90, 11, 103, 40], 'pd_gevtevxmm_simbadclasses': [4158, 190, 4667, 1012]}


Looking in separate catalogs:

In [60]:
#for regular_expression in regular_expressions:
#    print('Looking for : ' + regular_expression)
#    for j in separate_catalogs:
#        success, catalog_name, number_obj = find_number_in_catalogs(regular_expression, j)
#        if success:
#            resultats_of_search[catalog_name].append(number_obj)
#print('Done')

In [61]:
print(resultats_of_search)

{'tevsimbad': [], 'gevsimbad': [], 'gevtevsimbad': [], 'gev': [], 'tev': [], 'simbad': [], 'gevtev_simbadclasses': [90, 11, 103, 40], 'pd_gevtevxmm_simbadclasses': [4158, 190, 4667, 1012]}


In [62]:
pd_gevtevxmm_simbadclasses 

,gev_0FGL_Name,gev_1FGL_Name,gev_1FHL_Name,gev_2FGL_Name,gev_ASSOC1,gev_ASSOC2,gev_ASSOC_GAM1,gev_ASSOC_GAM2,gev_ASSOC_GAM3,gev_ASSOC_TEV,...,xmm_SC_HR3,xmm_SC_HR3_ERR,xmm_SC_HR4,xmm_SC_HR4_ERR,xmm_SC_POSERR,xmm_SC_RA,xmm_SC_SUM_FLAG,xmm_SC_VAR_FLAG,xmm_SRCID,xmm_WEBPAGE_URL
0,b' ',b' ',b' ',b' ',b'Tycho ',b' ',b' ',b' ',b' ',b'Tycho ',...,-0.750598,0.014265,-0.990890,0.013705,0.604278,6.312711,3,84,200962101010065,b'http://xmm-catalog.irap.omp.eu/source/200962...
1,b' ',b' ',b' ',b' ',b'Tycho ',b' ',b' ',b' ',b' ',b'Tycho ',...,-0.654271,0.018386,-0.998619,0.009212,0.569010,6.313853,3,84,200962101010062,b'http://xmm-catalog.irap.omp.eu/source/200962...
2,b' ',b' ',b' ',b' ',b'Tycho ',b' ',b' ',b' ',b' ',b'Tycho ',...,-0.834711,0.044069,-1.000000,0.030225,1.545560,6.318086,3,70,205111801010066,b'http://xmm-catalog.irap.omp.eu/source/205111...
3,b' ',b' ',b' ',b' ',b'Tycho ',b' ',b' ',b' ',b' ',b'Tycho ',...,-0.877829,0.022834,-0.999319,0.022730,0.605936,6.318439,3,84,200962101010075,b'http://xmm-catalog.irap.omp.eu/source/200962...
4,b' ',b' ',b' ',b' ',b'Tycho ',b' ',b' ',b' ',b' ',b'Tycho ',...,-1.000000,1.730460,1.000000,4.563300,2.010450,6.320361,3,0,204123803010008,b'http://xmm-catalog.irap.omp.eu/source/204123...
5,b' ',b' ',b' ',b' ',b'Tycho ',b' ',b' ',b' ',b' ',b'Tycho ',...,-0.612208,0.108985,-0.952844,0.106118,1.682760,6.320511,3,70,204123801010072,b'http://xmm-catalog.irap.omp.eu/source/204123...
6,b' ',b' ',b' ',b' ',b'Tycho ',b' ',b' ',b' ',b' ',b'Tycho ',...,-0.604622,0.043458,-0.994225,0.023340,1.525510,6.322434,3,70,205111801010015,b'http://xmm-catalog.irap.omp.eu/source/205111...
7,b' ',b' ',b' ',b' ',b'Tycho ',b' ',b' ',b' ',b' ',b'Tycho ',...,-0.527333,0.001039,-0.877867,0.001089,0.531535,6.323253,4,84,200962101010027,b'http://xmm-catalog.irap.omp.eu/source/200962...
8,b' ',b' ',b' ',b' ',b'Tycho ',b' ',b' ',b' ',b' ',b'Tycho ',...,-0.659082,0.015134,-0.999987,0.007768,0.562195,6.325806,3,84,200962101010070,b'http://xmm-catalog.irap.omp.eu/source/200962...
9,b' ',b' ',b' ',b' ',b'Tycho ',b' ',b' ',b' ',b' ',b'Tycho ',...,-1.000000,0.019862,1.000000,7.099610,0.929340,6.336368,3,70,203105902010063,b'http://xmm-catalog.irap.omp.eu/source/203105...


In [63]:
for i in resultats_of_search.keys():
    if i.find('simbad') != -1 and i.find('simbadclasses') == -1:
        continue
    print(i + ' :')
    display(name_to_catalog[i][name_to_column[i]].loc[resultats_of_search[i]])

gev :


,gev_2FGL_Name,gev_1FGL_Name,gev_1FHL_Name,gev_2FGL_Name,gev_ASSOC1,gev_ASSOC2,gev_ASSOC_GAM1,gev_ASSOC_GAM2,gev_ASSOC_GAM3,gev_ASSOC_TEV,gev_CLASS1,gev_GLAT,gev_GLON


tev :


,tev_common_name,tev_gamma_names,tev_fermi_names,tev_other_names,tev_classes,tev_glat,tev_glon


gevtev_simbadclasses :


,gev_2FGL_Name,gev_1FGL_Name,gev_1FHL_Name,gev_2FGL_Name,gev_ASSOC1,gev_ASSOC2,gev_ASSOC_GAM1,gev_ASSOC_GAM2,gev_ASSOC_GAM3,gev_ASSOC_TEV,...,tev_common_name,tev_gamma_names,tev_fermi_names,tev_other_names,tev_classes,tev_glat,tev_glon,s_class,s_simbad,s_MAIN_ID
90,b'2FGL J1826.3-1450 ',b'1FGL J1826.2-1450 ',b'1FHL J1826.2-1450 ',b'2FGL J1826.3-1450 ',b'LS 5039 ',b' ',b' ',b' ',b' ',b'LS 5039 ',...,b'LS 5039',b'HESS J1826-148',b'',b'RX J1826.2-1450',b'bin',-1.289216,16.881586,['B0Ve'],1,"[""b'V* V479 Sct'""]"
11,b'2FGL J0240.5+6113 ',b'1FGL J0240.5+6113 ',b'1FHL J0240.2+6113 ',b'2FGL J0240.5+6113 ',b'LS I+61 303 ',b' ',b'1AGL J0242+6111',b' ',b'EGR J0240+6112 ',b'LS I +61 303 ',...,b'LS I +61 303',b'VER J0240+612',b'',b'',b'bin',1.086135,135.675278,['B0Ve'],1,"[""b'LS I +61 303'"" ""b'LS I +61 303'""]"
103,b'2FGL J2032.2+4126 ',b'1FGL J2032.2+4127 ',b'1FHL J2032.1+4125 ',b'2FGL J2032.2+4126 ',b'LAT PSR J2032+4127 ',b' ',b'1AGL J2032+4102',b' ',b' ',b'TeV J2032+4130 ',...,b'MGRO J2031+41',b'',b'',"b'TeV J2032+4130,TeV 2032+413,0FGL J2032.2+4122'",b'unid',1.111117,80.299477,['B0Ve'],1,"[""b'[MT91] 213'"" ""b'Schulte 21'""]"
40,b'2FGL J1019.0-5856 ',b'1FGL J1018.6-5856 ',b'1FHL J1018.9-5855 ',b'2FGL J1019.0-5856 ',b'1FGL J1018.6-5856 ',b' ',b' ',b' ',b' ',b'HESS J1018-589 ',...,b'HESS J1018-589 A',b'HESS J1018-589 A',b'1FGL J1018.6-5856',"b'SNR G284.3-01.8,MSH 10-53'",b'bin',-1.779517,284.314087,['B0Ve'],2,"[""b'2FGL J1019.0-5856'""]"


pd_gevtevxmm_simbadclasses :


,gev_2FGL_Name,gev_1FGL_Name,gev_1FHL_Name,gev_2FGL_Name,gev_ASSOC1,gev_ASSOC2,gev_ASSOC_GAM1,gev_ASSOC_GAM2,gev_ASSOC_GAM3,gev_ASSOC_TEV,...,tev_gamma_names,tev_fermi_names,tev_other_names,tev_classes,tev_glat,tev_glon,xmm_IAUNAME,s_class,s_simbad,s_MAIN_ID
4158,b'2FGL J1826.3-1450 ',b'1FGL J1826.2-1450 ',b'1FHL J1826.2-1450 ',b'2FGL J1826.3-1450 ',b'LS 5039 ',b' ',b' ',b' ',b' ',b'LS 5039 ',...,b'HESS J1826-148',b'',b'RX J1826.2-1450',b'bin',-1.289216,16.881586,b'3XMM J182549.2-145024',['B0Ve'],1,"[""b'V* V479 Sct'""]"
190,b'2FGL J0240.5+6113 ',b'1FGL J0240.5+6113 ',b'1FHL J0240.2+6113 ',b'2FGL J0240.5+6113 ',b'LS I+61 303 ',b' ',b'1AGL J0242+6111',b' ',b'EGR J0240+6112 ',b'LS I +61 303 ',...,b'VER J0240+612',b'',b'',b'bin',1.086135,135.675278,b'3XMM J024005.3+611247',['B0Ve'],1,"[""b'LS I +61 303'"" ""b'LS I +61 303'""]"
4667,b'2FGL J2032.2+4126 ',b'1FGL J2032.2+4127 ',b'1FHL J2032.1+4125 ',b'2FGL J2032.2+4126 ',b'LAT PSR J2032+4127 ',b' ',b'1AGL J2032+4102',b' ',b' ',b'TeV J2032+4130 ',...,b'',b'',"b'TeV J2032+4130,TeV 2032+413,0FGL J2032.2+4122'",b'unid',1.111117,80.299477,b'3XMM J203145.8+412815',['B0Ve'],1,"[""b'[MT91] 213'"" ""b'Schulte 21'""]"
1012,b'2FGL J1019.0-5856 ',b'1FGL J1018.6-5856 ',b'1FHL J1018.9-5855 ',b'2FGL J1019.0-5856 ',b'1FGL J1018.6-5856 ',b' ',b' ',b' ',b' ',b'HESS J1018-589 ',...,b'HESS J1018-589 A',b'1FGL J1018.6-5856',"b'SNR G284.3-01.8,MSH 10-53'",b'bin',-1.779517,284.314087,b'3XMM J101826.8-585838',['B0Ve'],2,"[""b'2FGL J1019.0-5856'""]"


The following cell saves information about objects found.

In [64]:
#gevtevsimbad[gevtevs_columns].to_csv('resultat_gevtevsimbad.txt',sep = '\t')
#gevtev[tev_gev_columns].to_csv('resultat_gevtev.txt',sep = '\t')
#gevsimbad[s_gev_columns].to_csv('resultat_sgev.txt',sep = '\t')
#tevsimbad[s_tev_columns].to_csv('resultat_stev.txt',sep = '\t')

In [65]:
#for i in resultats_of_search.keys():
#    (name_to_catalog[i].loc[resultats_of_search[i]]).to_csv('resultats/resultat_' + i +'.txt',sep = '\t')

The following cell adds to bases used in the beginning a column that indicates the objects choosen in this ipynb.

In [66]:
for i in resultats_of_search.keys():
    print(i)
    if i in ['gev','tev','simbad']:
        continue
    catalog = name_to_catalog[i].copy()
    catalog.is_copy = False
    catalog['marked'] = 0
    (catalog.loc[resultats_of_search[i], 'marked'])= 1
    (catalog).to_csv('markeddata/' + i +'.txt',sep = '\t')


tevsimbad
gevsimbad
gevtevsimbad
gev
tev
simbad
gevtev_simbadclasses
pd_gevtevxmm_simbadclasses


/home/masha/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:4384: FutureWarning: Attribute 'is_copy' is deprecated and will be removed in a future version.
  object.__getattribute__(self, name)
/home/masha/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:4385: FutureWarning: Attribute 'is_copy' is deprecated and will be removed in a future version.
  return object.__setattr__(self, name, value)
